In [ ]:
from __future__ import division
import pandas as pd
import numpy as np
import time
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MaxNLocator

from matplotlib.ticker import FormatStrFormatter

from scipy import stats
import scipy.stats as sstat
import scipy.signal as ssig
import h5py
from mpl_toolkits.mplot3d import Axes3D
import os
from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA as sklearnPCA
import re

# import ephys_unit_analysis as ena
import mz_ephys_unit_analysis as mz_ena
import mz_unit_dur_freq as mz_udf

#import resampy
import fnmatch
import seaborn as sns
%matplotlib inline
%load_ext autoreload
%autoreload 2
mpl.rcParams['pdf.fonttype'] = 42 
mpl.rcParams['font.sans-serif']=['Arial', 'Helvetica','Bitstream Vera Sans', 'DejaVu Sans', 'Lucida Grande', 
                                 'Verdana', 'Geneva', 'Lucid', 'Avant Garde', 'sans-serif']  

sns.despine()

In [2]:
# for publication quality plots
def set_pub_plots(pal=sns.blend_palette(["gray","crimson", 'cyan', 'magenta', 'purple'  ],5)):
    sns.set_style("white")
    sns.set_palette(pal)
    sns.set_context("poster", font_scale=1.5, rc={"lines.linewidth": 2.5, "axes.linewidth":2.5, 'figure.facecolor': 'white'}) 
    sns.set_style("ticks", {"xtick.major.size": 8, "ytick.major.size": 8})
    plt.rcParams['axes.linewidth'] = 2.5

rc_pub={'font.size': 25, 'axes.labelsize': 25, 'legend.fontsize': 25.0, 
        'axes.titlesize': 25, 'xtick.labelsize': 25, 'ytick.labelsize': 25, 
        'axes.linewidth':2.5, 'lines.linewidth': 2.5,
        'xtick.color': 'black', 'ytick.color': 'black', 'axes.edgecolor': 'black',
        'axes.labelcolor':'black','text.color':'black'}
# to restore the defaults, call plt.rcdefaults() 

set_pub_plots()

In [ ]:
pal=sns.blend_palette(['black','royalblue'], 2)
sns.palplot(pal)
sns.set_palette(pal)

---

# First, load in the previously saved dataFrame

In [ ]:
final_df = pd.read_pickle(r"D:\mz_Data\saved_dfs\HPC_nmda\psth_df.pkl")

print(final_df.et.nunique())
print(final_df.group.unique())
print(final_df.region.unique())
print(final_df.situ.unique())
print(final_df.situ.unique())

final_df.head()

---

### Now choose the reward situation you want
This requires a __`user input`__ to run

In [ ]:

rew_to_plot = input('What reward training stage (pre, post, novel): ')
if rew_to_plot == 'pre':
    reward_df = final_df[final_df['situ'] == rew_to_plot]
    plt_title = 'Pre training'
elif rew_to_plot == 'post':
    reward_df = final_df[final_df['situ'] == rew_to_plot]
    plt_title = 'Post training'
elif rew_to_plot == 'novel':
    reward_df = final_df[final_df['situ'] == rew_to_plot]
    plt_title = 'Novel'
V1_all_groups_df = reward_df[reward_df['region'] == 'v1']
V1_A = reward_df[(reward_df['region'] == 'v1') & (reward_df['group'] == 'sham')]
V1_B = reward_df[(reward_df['region'] == 'v1') & (reward_df['group'] == 'nmda')]


---

# Selecting only the visually responsive units
### Separating by group and moving to arrays

In [52]:
def vis_resp_units(final_df, region='v1', time_window=[0.5, 0.9], z_score = 1.0):
    
    test_df = final_df[final_df['region'] == region]
    
    vis_resp = test_df[(test_df['times'] >= time_window[0]) 
                       & (test_df['times'] <= time_window[1]) 
                       & (test_df['zscore'] > z_score)]
    
    test2 = test_df[test_df.cuid.isin(vis_resp.cuid.unique())]
    
    return test2
    

In [ ]:
vis_resp_df = vis_resp_units(V1_all_groups_df, region='v1')

plot_A = vis_resp_df[vis_resp_df['group'] == 'sham']
plot_B = vis_resp_df[vis_resp_df['group'] == 'nmda']

A_array = np.array(plot_A.pivot(index='cuid', columns='times', values='zscore'))
print(A_array.shape) #this is [units x times] and the values are above

B_array = np.array(plot_B.pivot(index='cuid', columns='times', values='zscore'))
print(B_array.shape) #this is [units x times] and the values are above

---

# Quantifying the oscillation duration and frequency

In [54]:
durationA = []
indA = []
for unit in range(A_array.shape[0]):
    yy = A_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    durationA.append(dur)
    indA.append(peak_ind)

durationB = []
indB = []
for unit in range(B_array.shape[0]):
    yy = B_array[unit]
    dur, peak_ind = mz_udf._duration_peaks_unit(yy)
    durationB.append(dur)
    indB.append(peak_ind)

In [55]:
indA_array = np.concatenate(indA, axis=0)
indB_array = np.concatenate(indB, axis=0)

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 12.0 # x dimension
fig_size[1] = 5.0 # y dimension

sns.histplot(indA_array, bins=140, linewidth=0, stat='density', color='black')
sns.histplot(indB_array, bins=140, linewidth=0, stat='density', color='royalblue')
# sns.kdeplot(indA_array, bw_method=0.1, color='cyan')
# sns.kdeplot(indB_array, bw_method=0.1, color='magenta')

plt.title(str('peak times: '+plt_title))
plt.xlim([40,150])
plt.xticks(ticks=[40,60,80,100,120,140], labels=[0.4,0.6,0.8,1.0,1.2,1.4])
plt.yticks(ticks=[0,0.03,0.06,0.09])
plt.xlabel('Time (s)')
plt.axvspan(50,70, color='grey', alpha=0.2)

plt.show()

In [ ]:
sns.kdeplot(indA_array, bw_method=0.1, color='black', fill=True)
sns.kdeplot(indB_array, bw_method=0.1, color='royalblue', fill=True)

plt.title(str('kde peak times: '+ plt_title))
plt.xlim([40,150])
plt.xticks(ticks=[40,60,80,100,120,140], labels=[0.4,0.6,0.8,1.0,1.2,1.4])
plt.yticks(ticks=[0,0.02,0.04])
plt.xlabel('Time (s)')
plt.axvspan(50,70, color='grey', alpha=0.2)

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\dur_freq\novel_hist_kde.pdf", transparent=True)

plt.show()

---

# Oscillation duration

In [ ]:
durA_df = pd.DataFrame(tuple(zip(durationA,(['sham']*len(durationA)))),columns=['dur','group'])
durB_df = pd.DataFrame(tuple(zip(durationB,(['nmda']*len(durationB)))),columns=['dur','group'])
all_dur_df=pd.concat([durA_df,durB_df])
all_dur_df.head()

In [ ]:
#removes the NaN values from the above dataframe
all_dur_df=all_dur_df[~all_dur_df.dur.isna()]
all_dur_df.shape

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
sns.kdeplot(data=all_dur_df, x="dur", 
            hue="group",
            cumulative=True, 
            common_norm=False,
            common_grid=True,
            bw_method=0.5,
            legend=False
           )

plt.title(plt_title)
plt.xlabel('Time (s)')
plt.xticks([0,0.2,0.4,0.6,0.8,1.0])
plt.xlim([-0.1,1.1])
plt.yticks([0,0.25,0.5,0.75,1.0])

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\dur_freq\dur_novel_kde.pdf", transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 2.0 # x dimension
fig_size[1] = 5.0 # y dimension

ax1=plt.subplot(111)
sns.barplot(data=all_dur_df, x='group', y='dur', order=["sham","nmda"], ci=68)

plt.ylabel('Time (s)')
plt.xlabel('')
plt.yticks(ticks=[0,0.1,0.2,0.3,0.4,0.5])
plt.ylim([0,0.55])
plt.title(plt_title)

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\dur_freq\dur_novel_bar.pdf", transparent=True)

plt.show()

# Run the stats on the oscillation duration

In [ ]:
result = sstat.ks_2samp(all_dur_df[all_dur_df['group']=='sham'].dur, all_dur_df[all_dur_df['group']=='nmda'].dur)
print('Oscillation Duration')
print(plt_title)
print('sham units: {0} --- nmda units: {1}'.format(len(indA),len(indB)))
print(result)

In [ ]:
print('Oscillation Duration')
print(plt_title)

print('Mean of sham: {0}'.format(np.mean(all_dur_df[all_dur_df['group']=='sham'].dur)))
print('Mean of nmda: {0}'.format(np.mean(all_dur_df[all_dur_df['group']=='nmda'].dur)))

result = sstat.ttest_ind(all_dur_df[all_dur_df['group']=='sham'].dur, all_dur_df[all_dur_df['group']=='nmda'].dur)
print(result)

---

# Quantify the unit oscillation frequency
### Looking at the 2nd to 3rd peaks frequency

In [64]:
mean_freqA = [1/np.diff(i)*100+1 for i in indA]
mean_freqB = [1/np.diff(i)*100+1 for i in indB]

In [ ]:
freqA_df = pd.DataFrame(tuple(zip(mean_freqA,(['sham']*len(mean_freqA)))),columns=['freq','group'])
freqB_df = pd.DataFrame(tuple(zip(mean_freqB,(['nmda']*len(mean_freqB)))),columns=['freq','group'])
all_freq_df=pd.concat([freqA_df,freqB_df])

all_freq_df['up_cyc'] = all_freq_df.freq.str[1] #this is the 2-3 peak cycle
# all_freq_df['up_cyc2'] = all_freq_df.freq.str[2] #this is the 3-4 peak cycle

all_freq_df.head()

In [ ]:
# removes the NaN values from the above dataframe
all_freq_df=all_freq_df[~all_freq_df.up_cyc.isna()]
# all_freq_df=all_freq_df[~all_freq_df.up_cyc2.isna()]
all_freq_df.shape

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
# sns.histplot(data=all_freq_df, x="up_cyc", hue="group")
sns.kdeplot(data=all_freq_df, x="up_cyc", hue="group",
            cumulative=False, common_norm=False, bw_method=0.7, fill=True, legend=False)

plt.title(plt_title)
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,13])
plt.ylim([0,0.35])
plt.yticks([0,0.1,0.2,0.3])

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\dur_freq\freq_novel_kde.pdf", transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
sns.kdeplot(data=all_freq_df, x="up_cyc", hue="group",
            cumulative=True, common_norm=False,
            bw_method=0.7,legend=False)

plt.title(plt_title)
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,13])

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\dur_freq\freq_novel_cde.pdf", transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 2.0 # x dimension
fig_size[1] = 5.0 # y dimension

ax1=plt.subplot(111)
sns.barplot(data=all_freq_df, x='group', y='up_cyc', order=["sham","nmda"], ci=68)

plt.ylabel('Frequency (Hz)')
plt.xlabel('')
ax1.set_xticklabels(['sham','nmda'])
plt.yticks(ticks=[0,2,4,6])
plt.ylim([0,6.5])
plt.title(plt_title)

# Change the end of this next line with a new file name!!
# plt.savefig(r"C:\Users\AChub_Lab\Desktop\tmp_nmda\units\dur_freq\freq_novel_bar.pdf", transparent=True)

plt.show()

In [ ]:
# upper-cycles frequency

result = sstat.ks_2samp(all_freq_df[all_freq_df['group']=='sham'].up_cyc, 
                        all_freq_df[all_freq_df['group']=='nmda'].up_cyc)

print('Oscillation Frequency (upper cycles)')
print('sham: {0} --- nmda: {1}'.format(all_freq_df[all_freq_df['group']=='sham'].up_cyc.nunique(),
                                   all_freq_df[all_freq_df['group']=='nmda'].up_cyc.nunique()))
print(plt_title)
print(result)

In [ ]:
# upper-cycles frequency

print('Oscillation Frequency (upper cycles)')
print('Mean of sham: {0}'.format(np.mean(all_freq_df[all_freq_df['group']=='sham'].up_cyc)))
print('Mean of nmda: {0}'.format(np.mean(all_freq_df[all_freq_df['group']=='nmda'].up_cyc)))

result = sstat.ttest_ind(all_freq_df[all_freq_df['group']=='sham'].up_cyc, 
                         all_freq_df[all_freq_df['group']=='nmda'].up_cyc)
print(plt_title)
print(result)

---

---

---

---

### Looking at the mean oscillation frequency - original analysis method

In [ ]:
mean_freqA = [np.mean(1/np.diff(i)*100)+1 for i in indA]
mean_freqB = [np.mean(1/np.diff(i)*100)+1 for i in indB]

# old, would stack all of the freq, not the mean freq per unit
# freqA=np.hstack([1/np.diff(i)*100 for i in indA])
# freqB=np.hstack([1/np.diff(i)*100 for i in indB])

In [ ]:
freqA_df = pd.DataFrame(tuple(zip(mean_freqA,(['WT']*len(mean_freqA)))),columns=['freq','group'])
freqB_df = pd.DataFrame(tuple(zip(mean_freqB,(['DKO']*len(mean_freqB)))),columns=['freq','group'])
all_freq_df=pd.concat([freqA_df,freqB_df])
all_freq_df.head()

In [ ]:
#removes the NaN values from the above dataframe
all_freq_df=all_freq_df[~all_freq_df.freq.isna()]
all_freq_df.shape

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
# sns.histplot(data=all_freq_df, x="freq", hue="group")
sns.kdeplot(data=all_freq_df, x="freq", hue="group",
            cumulative=False, common_norm=False, bw_method=1, fill=True, legend=False)

plt.title(plt_title)
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,13])
plt.ylim([0,0.25])

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\units\dur_freq\frequency\novel_hist_kde.pdf", transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 7.0 # x dimension
fig_size[1] = 4.0 # y dimension

ax1= plt.subplot(111)
sns.kdeplot(data=all_freq_df, x="freq", hue="group",
            cumulative=True, common_norm=False,
            bw_method=0.7,legend=False)

plt.title(plt_title)
plt.xlabel('Frequency (Hz)')
plt.xticks([0,2,4,6,8,10,12])
plt.xlim([-1,13])

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\units\dur_freq\frequency\novel_kde.pdf", transparent=True)

plt.show()

In [ ]:
fig_size = plt.rcParams["figure.figsize"]
fig_size[0] = 2.0 # x dimension
fig_size[1] = 5.0 # y dimension

ax1=plt.subplot(111)
sns.barplot(data=all_freq_df, 
            x='group',
            y='freq',
            order=["WT","DKO"],
            ci=68,
           )
plt.ylabel('Frequency (Hz)')
plt.xlabel('')
ax1.set_xticklabels(['WT','DKO'])
plt.yticks(ticks=[0,2,4,6])
plt.ylim([0,6.5])
plt.title(plt_title)

# Change the end of this next line with a new file name!!
# plt.savefig(r"D:\mz_Data\DATA_Figs\HDAC\units\dur_freq\frequency\novel_bar.pdf", transparent=True)

plt.show()

---

# Run the stats on the oscillation frequency

### 2-sample KS test for the cumulative distribution line


In [ ]:
# mean of all cycles frequency

result = sstat.ks_2samp(all_freq_df[all_freq_df['group']=='WT'].freq, 
                        all_freq_df[all_freq_df['group']=='DKO'].freq)

print('Oscillation Frequency (mean)')
print('WT: {0} --- DKO: {1}'.format(all_freq_df[all_freq_df['group']=='WT'].freq.nunique(),
                                   all_freq_df[all_freq_df['group']=='DKO'].freq.nunique()))
print(plt_title)
print(result)

### Welch's t-test on the mean

In [ ]:
# mean of all cycles frequency


print('Oscillation Frequency (mean)')
print('Mean of WT: {0}'.format(np.mean(all_freq_df[all_freq_df['group']=='WT'].freq)))
print('Mean of DKO: {0}'.format(np.mean(all_freq_df[all_freq_df['group']=='DKO'].freq)))

result = sstat.ttest_ind(all_freq_df[all_freq_df['group']=='WT'].freq, 
                         all_freq_df[all_freq_df['group']=='DKO'].freq)
print(plt_title)
print(result)